# Сессия 1

In [1]:
import pandas as pd 
import numpy as nmp
import pdfplumber
import warnings
warnings.filterwarnings("ignore")
import pdfplumber
from tabula import read_pdf
import re 

### Парсинг

In [2]:
import os

In [3]:
directory = "C:\\Users\\User\\Desktop\\COMP\\pdf"

Получаем список пдф файлов из директории 

In [4]:
comps = os.listdir(directory)

In [5]:
comps

['Магистральные линии связи. Строительство и эксплуатация ВОЛП.pdf',
 'Медицинская оптика.pdf',
 'Метрология.pdf',
 'Мехатроника .pdf',
 'Мобильная робототехника.pdf',
 'Обработка листового металла.pdf',
 'Обслуживание грузовой техники.pdf',
 'Обслуживание и ремонт оборудования релейной защиты и автоматики.pdf',
 'Огранка ювелирных вставок.pdf',
 'Организация строительного производства.pdf',
 'Охрана окружающей среды.pdf',
 'Охрана труда.pdf',
 'Плотницкое дело(юниоры).pdf',
 'Плотницкое дело.pdf',
 'Полимеханика и автоматизация.pdf',
 'Правоохранительная деятельность (Полицейский).pdf',
 'Преподавание музыки в школе.pdf',
 'Производственная сборка изделий авиационной техники.pdf',
 'Производство металлоконструкций.pdf',
 'Промышленный дизайн .pdf',
 'Сметное дело.pdf',
 'Спасательные работы.pdf',
 'Столярное дело.pdf',
 'Туроператорская деятельность.pdf',
 'Укладка напольных покрытий.pdf',
 'Управление локомотивом.pdf',
 'Управление экскаватором.pdf',
 'Холодильная техника и системы к

Две функции для парсинга одного пдф файла, тк они разные то мы будем использовать сначала первую, а потом, если не сработает, то вторую. 

In [6]:
def one_file_table(filename, pg):
    L=[]
    pdf = pdfplumber.open(filename)
    for i in range(pg):
        page = pdf.pages[i]
        try:
            table = page.extract_table()
            df = pd.DataFrame(table)
            L.append(df)
        except Exception:
            print("n", end = ",")
    df = pd.concat(L,ignore_index=True)
    df.columns = ["Раздел", "Слова", "Важность"]
    df = df.drop([0,1,2]).reset_index(drop=True)
    return df

In [7]:
def one_file_table_2(filename, pg):
    L=[]
    pdf = pdfplumber.open(filename)
    for i in range(pg):
        page = pdf.pages[i]
        try:
            table = page.extract_table()
            df = pd.DataFrame(table)

            if df.shape[0] > 1:
                df = df.drop([0,2,5,6,8], axis=1)
                for i in range(len(df)):
                    if df.iloc[i][4] != None:
                        df.iloc[i][3] = df.iloc[i][4]
                df = df.drop([4], axis=1)
                L.append(df)
        except Exception:
            print("n", end = ",")
    df= pd.concat(L)    
    df.columns = ["Раздел", "Слова", "Важность"]
    df= df.drop([0,1]).reset_index(drop=True)
    return df

В цикле проходим по всем файлам и пытаемся спарсить данные, если это не получается то название файла записывается в массив FFF

In [8]:
FFF=[]
DF=[]
for fname in comps:
    filepath = directory + "\\"+fname
    try:
        df = one_file_table(filepath, 14)
    except Exception:
        try:
            df = one_file_table_2(filepath, 14)
        except Exception:
            try:
                df = one_file_table(filepath, 11)
            except Exception:
                try:
                    df = one_file_table_2(filepath, 11)
                except Exception:
                    print(fname)
                    FFF.append(fname)
                    continue
    df["Компетенция"] = fname[:fname.find(".")]
    DF.append(df)

Магистральные линии связи. Строительство и эксплуатация ВОЛП.pdf
n,n,n,n,n,n,Мехатроника .pdf
n,n,n,n,n,n,n,n,n,n,Обслуживание и ремонт оборудования релейной защиты и автоматики.pdf
n,n,n,n,n,n,n,n,Охрана труда.pdf
Полимеханика и автоматизация.pdf
n,n,n,n,n,Правоохранительная деятельность (Полицейский).pdf
n,n,n,n,n,n,n,n,n,n,n,n,n,n,Туроператорская деятельность.pdf
n,n,n,n,n,n,n,n,n,n,n,Управление локомотивом.pdf
n,n,n,n,n,n,Управление экскаватором.pdf
n,n,n,n,n,n,n,n,n,n,Экспедирование грузов.pdf


In [9]:
df = pd.concat(DF)

In [10]:
df

,Раздел,Слова,Важность,Компетенция
0,1,"Организовывать, планировать и анализировать ра...",7,Медицинская оптика
1,None,Специалист должен знать и понимать: \nназначен...,None,Медицинская оптика
2,None,Специалист должен уметь: \nподготавливать и по...,None,Медицинская оптика
3,2,Организовывать сбыт продукции,6,Медицинская оптика
4,None,Специалист должен знать и понимать: \n• основ...,None,Медицинская оптика
...,...,...,...,...
9,None,Специалист должен знать и понимать: \n• знани...,None,Холодильная техника и системы кондиционирования
10,None,Специалист должен уметь: \n• выполнять заправ...,None,Холодильная техника и системы кондиционирования
11,None,Специалист должен знать и понимать: \n• элект...,None,Холодильная техника и системы кондиционирования
12,None,Специалист должен уметь: \n• проверять и тест...,None,Холодильная техника и системы кондиционирования


In [11]:
len(FFF)

10

По итогу у нас есть датасет из 351 строчки и 4 столбцов где имеется название компетенции, слова её описывающие и важность.
Из 30 имеющихся пдф файлов получилось спарсить 20. Сохраним результат в parsedData.csv

In [12]:
df.to_csv("parsedData.csv")

### Работа с excel данными 

In [13]:
data = pd.read_excel("dataset.xlsx")

In [14]:
(data.describe().append(data.isnull().sum().rename("isnull")))#.iloc[-1]

,19,22,25,28,1,2,3,4,5,6,...,20,21,23,24,26,27,29,30,31,32
count,96.000000,60.000000,22.000000,12.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,17.668750,16.916667,9.454545,10.833333,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,13.596403,16.191954,2.685539,3.996210,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,3.000000,3.000000,3.000000,5.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,10.000000,8.750000,10.000000,7.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,15.000000,13.000000,10.000000,10.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,20.250000,20.000000,10.000000,15.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,100.000000,100.000000,13.000000,15.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
isnull,34.000000,70.000000,108.000000,118.000000,0.0,0.0,0.0,1.0,0.0,0.0,...,34.0,70.0,71.0,108.0,108.0,118.0,118.0,124.0,126.0,126.0


In [15]:
data.head(1)

,1,2,3,4,5,6,7,8,9,10,...,23,24,25,26,27,28,29,30,31,32
0,Разработка компьютерных игр и мультимедийных п...,Разработка компьютерных игр и мультимедийных п...,Организация и управление работой,13,Специалист должен знать и понимать: • Принципы...,Программирование на языке С#,27,Специалист должен знать и понимать: • Базовые ...,Unity 3D,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Добавим данные из excel таблицы к данным которые мы спарсили. В Экселе в первой колонке содержатся названия компетенций, в остальных ячейках в этой же строчке содержаться слова пренадлежащие данной компетенции и числа с важностью. если после ячейки в которой есть слова идет число, то эти слова это начало новой подтемы и это число является её важностью.<br><br>Выполнив полный перебор всех ячеек по строчкам и приведя к такому же виду, как и в датасете со спарсиными данными, добавим эти данные

In [17]:
N = data.iloc[0][21]
for j in range(data.shape[0]):
    Comp = data.iloc[j][1]
    for i in range(2,len(data.iloc[j])-1):
        tmp = {"Раздел":N,"Слова":N,"Важность":N,"Компетенция":Comp}
        if type(data.iloc[j][i]) == int or type(data.iloc[j][i]) == float:
            continue
        if data.iloc[j][i] != N and (type(data.iloc[j][i]) != int or type(data.iloc[j][i]) != float):
            tmp["Слова"] = data.iloc[j][i]
        if type(data.iloc[j][i+1]) == int or type(data.iloc[j][i+1]) == float:
            tmp["Важность"] = data.iloc[j][i+1]
        df = df.append(tmp, ignore_index=True)

In [18]:
dr=[]
for i in range(df.shape[0]):
    if type(df.iloc[i]['Слова']) == float:
        df.loc[i-1]['Важность'] = df.iloc[i]['Слова']
        dr.append(i)

In [19]:
df.drop(dr,inplace=True)

In [20]:
df = df[df['Слова'].notna()]

In [21]:
df = df.reset_index(drop=True)

In [22]:
df["Важность"] = df["Важность"].fillna(0)

In [23]:
v = int(df.iloc[0]["Важность"])
for i in range(1, df.shape[0]):
    if df.loc[i]["Важность"] != 0:
        v = df.loc[i]["Важность"]
    else:
        df.loc[i]["Важность"] = v

##### Получим

In [24]:
df

,Раздел,Слова,Важность,Компетенция
0,1,"Организовывать, планировать и анализировать ра...",7,Медицинская оптика
1,None,Специалист должен знать и понимать: \nназначен...,7,Медицинская оптика
2,None,Специалист должен уметь: \nподготавливать и по...,7,Медицинская оптика
3,2,Организовывать сбыт продукции,6,Медицинская оптика
4,None,Специалист должен знать и понимать: \n• основ...,6,Медицинская оптика
...,...,...,...,...
2141,NaN,Специалист должен знать и понимать: • Способы ...,20,Ювелирное дело
2142,NaN,Изготовление сложных элементов и ювелирных изд...,20,Ювелирное дело
2143,NaN,Специалист должен знать и понимать: • Способы ...,20,Ювелирное дело
2144,NaN,Обработка поверхности,15,Ювелирное дело


Теперь наши слова необходимо преобразовать и почистить от мусора: уберем все символы, спецсимволы, числа. Добавим массив с русскими стопсловами и уберем их из наших данных. Дропнем колонку раздел, тк она не несет никакой смысловой нагрузки

In [30]:
for i in range(df.shape[0]):
    df["Слова"].iloc[i]  = re.sub('[–\n\r."("")"©•,«»:%$!@#0-9$a-z]', '', df["Слова"].iloc[i]).lower()

In [31]:
stopwords = ["тд","c","а","алло","без","белый","близко","более","больше","большой","будем","будет","будете","будешь","будто","буду","будут","будь","бы","бывает","бывь","был","была","были","было","быть","в","важная","важное","важные","важный","вам","вами","вас","ваш","ваша","ваше","ваши","вверх","вдали","вдруг","ведь","везде","вернуться","весь","вечер","взгляд","взять","вид","видел","видеть","вместе","вне","вниз","внизу","во","вода","война","вокруг","вон","вообще","вопрос","восемнадцатый","восемнадцать","восемь","восьмой","вот","впрочем","времени","время","все","все еще","всегда","всего","всем","всеми","всему","всех","всею","всю","всюду","вся","всё","второй","вы","выйти","г","где","главный","глаз","говорил","говорит","говорить","год","года","году","голова","голос","город","да","давать","давно","даже","далекий","далеко","дальше","даром","дать","два","двадцатый","двадцать","две","двенадцатый","двенадцать","дверь","двух","девятнадцатый","девятнадцать","девятый","девять","действительно","дел","делал","делать","делаю","дело","день","деньги","десятый","десять","для","до","довольно","долго","должен","должно","должный","дом","дорога","друг","другая","другие","других","друго","другое","другой","думать","душа","е","его","ее","ей","ему","если","есть","еще","ещё","ею","её","ж","ждать","же","жена","женщина","жизнь","жить","за","занят","занята","занято","заняты","затем","зато","зачем","здесь","земля","знать","значит","значить","и","иди","идти","из","или","им","имеет","имел","именно","иметь","ими","имя","иногда","их","к","каждая","каждое","каждые","каждый","кажется","казаться","как","какая","какой","кем","книга","когда","кого","ком","комната","кому","конец","конечно","которая","которого","которой","которые","который","которых","кроме","кругом","кто","куда","лежать","лет","ли","лицо","лишь","лучше","любить","люди","м","маленький","мало","мать","машина","между","меля","менее","меньше","меня","место","миллионов","мимо","минута","мир","мира","мне","много","многочисленная","многочисленное","многочисленные","многочисленный","мной","мною","мог","могу","могут","мож","может","может быть","можно","можхо","мои","мой","мор","москва","мочь","моя","моё","мы","на","наверху","над","надо","назад","наиболее","найти","наконец","нам","нами","народ","нас","начала","начать","наш","наша","наше","наши","не","него","недавно","недалеко","нее","ней","некоторый","нельзя","нем","немного","нему","непрерывно","нередко","несколько","нет","нею","неё","ни","нибудь","ниже","низко","никакой","никогда","никто","никуда","ним","ними","них","ничего","ничто","но","новый","нога","ночь","ну","нужно","нужный","нх","о","об","оба","обычно","один","одиннадцатый","одиннадцать","однажды","однако","одного","одной","оказаться","окно","около","он","она","они","оно","опять","особенно","остаться","от","ответить","отец","откуда","отовсюду","отсюда","очень","первый","перед","писать","плечо","по","под","подойди","подумать","пожалуйста","позже","пойти","пока","пол","получить","помнить","понимать","понять","пор","пора","после","последний","посмотреть","посреди","потом","потому","почему","почти","правда","прекрасно","при","про","просто","против","процентов","путь","пятнадцатый","пятнадцать","пятый","пять","работа","работать","раз","разве","рано","раньше","ребенок","решить","россия","рука","русский","ряд","рядом","с","с кем","сам","сама","сами","самим","самими","самих","само","самого","самой","самом","самому","саму","самый","свет","свое","своего","своей","свои","своих","свой","свою","сделать","сеаой","себе","себя","сегодня","седьмой","сейчас","семнадцатый","семнадцать","семь","сидеть","сила","сих","сказал","сказала","сказать","сколько","слишком","слово","случай","смотреть","сначала","снова","со","собой","собою","советский","совсем","спасибо","спросить","сразу","стал","старый","стать","стол","сторона","стоять","страна","суть","считать","т","та","так","такая","также","таки","такие","такое","такой","там","твои","твой","твоя","твоё","те","тебе","тебя","тем","теми","теперь","тех","то","тобой","тобою","товарищ","тогда","того","тоже","только","том","тому","тот","тою","третий","три","тринадцатый","тринадцать","ту","туда","тут","ты","тысяч","у","увидеть","уж","уже","улица","уметь","утро","хороший","хорошо","хотел бы","хотеть","хоть","хотя","хочешь","час","часто","часть","чаще","чего","человек","чем","чему","через","четвертый","четыре","четырнадцатый","четырнадцать","что","чтоб","чтобы","чуть","шестнадцатый","шестнадцать","шестой","шесть","эта","эти","этим","этими","этих","это","этого","этой","этом","этому","этот","эту","я","являюсь"]

In [32]:
df['Чистые слова'] = df['Слова'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

In [33]:
df.drop("Раздел", axis=1, inplace=True)

### Получим

In [34]:
df

,Слова,Важность,Компетенция,Чистые слова
0,организовывать планировать и анализировать работу,7,Медицинская оптика,организовывать планировать анализировать работу
1,специалист должен знать и понимать назначение ...,7,Медицинская оптика,специалист назначение использование техническо...
2,специалист должен уметь подготавливать и подде...,7,Медицинская оптика,специалист подготавливать поддерживать рабочее...
3,организовывать сбыт продукции,6,Медицинская оптика,организовывать сбыт продукции
4,специалист должен знать и понимать основные ...,6,Медицинская оптика,специалист основные торговые марки производите...
...,...,...,...,...
2141,специалист должен знать и понимать способы ис...,20,Ювелирное дело,специалист способы использования различных эле...
2142,изготовление сложных элементов и ювелирных изд...,20,Ювелирное дело,изготовление сложных элементов ювелирных издел...
2143,специалист должен знать и понимать способы ис...,20,Ювелирное дело,специалист способы использования различных сло...
2144,обработка поверхности,15,Ювелирное дело,обработка поверхности


Сохраним в fullData.csv

In [35]:
df.to_csv("fullData.csv")

Подготовим Словарь данных для итогового датасета fullData

In [36]:
data = df.copy()
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
cvData = cv.fit_transform(data["Чистые слова"]).toarray()


In [37]:
cvDF = pd.DataFrame(data=cvData, columns=cv.get_feature_names())

Сохраним как СловарьДанных.csv

In [39]:
cvDF.to_csv("СловарьДанных.csv")